In [38]:
import librosa
import librosa.display
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [68]:
# Load the audio file
file_repo = '../CMR_subset_1.0/audio'

# extract features from all the files in the repo
def extract_features(file_repo):
  features = []
  for file in os.listdir(file_repo):
    if file.endswith(".wav"):
      file_path = os.path.join(file_repo, file)
      y, sr = librosa.load(file_path)
      chroma = librosa.feature.chroma_stft(y=y, sr=sr)
      mfccs = librosa.feature.mfcc(y=y, sr=sr)
      combined_features = np.vstack((chroma, mfccs)).T
      features.append(combined_features)
  return features

extracted_features = extract_features(file_repo)

In [59]:
file_path = '../CMR_subset_1.0/CMRdataset.csv'

def extract_labels(file_path):
  labels = []
  with open(file_path, 'r') as f:
    # leave out the header
    next(f)
    for line in f:
      line = line.strip()
      line = line.split(',')
      labels.append(line[-1])
  return labels

extracted_labels = extract_labels(file_path)

In [71]:
extracted_features = np.vstack(extracted_features)

print(extracted_features.shape)

(606733, 32)


### CNN Model for Rhythmic beat classification

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Assuming input shape is (5168, 32)
input_shape = (5168, 32)

# Define the CNN model
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
    self.pool = nn.MaxPool1d(kernel_size=2)
    self.fc1 = nn.Linear(64 * 2582, 64)
    self.fc2 = nn.Linear(64, 4)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = x.view(-1, 64 * 2582)  # Adjust the view size based on your specific task
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# Create an instance of the model
model = SimpleCNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Assuming your features and labels are NumPy arrays
features_tensor = torch.Tensor(extracted_features)
labels_tensor = torch.Tensor(extracted_labels).int()

# Create a DataLoader for handling batches
dataset = TensorDataset(features_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
  for inputs, labels in dataloader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

# Print the model architecture
print(model)

ValueError: expected sequence of length 5168 at dim 1 (got 4180)